In [1]:
import pandas as pd
import risk_kit as rk
%load_ext autoreload
%autoreload 2

In [3]:
rk.bond_price(10,1000,0,1,0.05)

0    613.913254
dtype: float64

In [57]:
import numpy as np

# Constants
face_value = 1000
yield_curve = 0.05  # 5% flat yield curve assumption

# Bond B3 characteristics (Zero-Coupon Bond)
maturity_B3 = 10
coupon_rate_B3 = 0  # 0% coupon rate
coupons_per_year_B3 = 1

# Function to calculate present value of cash flows for a bond
def bond_price(face_value, coupon_rate, coupons_per_year, maturity, discount_rate):
    coupon_amt = face_value * coupon_rate / coupons_per_year
    coupon_times = np.arange(1, maturity * coupons_per_year + 1)
    coupon_pv = np.sum(coupon_amt / (1 + discount_rate / coupons_per_year) ** coupon_times)
    face_pv = face_value / (1 + discount_rate / coupons_per_year) ** (maturity * coupons_per_year)
    price = coupon_pv + face_pv
    return price

# Calculate the price of bond B3 (Zero-Coupon Bond)
price_B3 = bond_price(face_value, coupon_rate_B3, coupons_per_year_B3, maturity_B3, yield_curve)

print(f"The price of bond B3 is: ${price_B3:.2f}")


The price of bond B3 is: $613.91


In [32]:
flow=rk.bond_cash_flows(5,1000,0.06,4)
duarion_B2 = rk.macaulay_duration(flow, 0.05/4)/4
duarion_B2

4.373363222636413

In [34]:
flow=rk.bond_cash_flows(15,1000,0.05,2)
duarion_B1 = rk.macaulay_duration(flow, 0.05/2)/2
duration_B1

18.745184153809944

In [35]:
flow=rk.bond_cash_flows(10,1000,0,1)
duarion_B3 = rk.macaulay_duration(flow, 0.05)
duration_B3

nan

In [37]:
import numpy as np
import pandas as pd

# Example liabilities
liabilities = pd.Series([-100000, -200000, -300000], index=[3, 5, 10])

# Discount rate (flat yield assumption of 5%)
discount_rate = 0.05

# Calculate Macaulay Duration
duration = rk.macaulay_duration(liabilities, discount_rate)
print(duration)


6.750917852744651


In [41]:


# Constants
face_value = 1000
yield_curve = 0.05  # 5% flat yield curve assumption

# Bond B1 characteristics
maturity_B1 = 15
coupon_rate_B1 = 0.05
coupons_per_year_B1 = 2

# Bond B2 characteristics
maturity_B2 = 5
coupon_rate_B2 = 0.06
coupons_per_year_B2 = 4

# Bond B3 characteristics (Zero-Coupon Bond)
maturity_B3 = 10
coupon_rate_B3 = 0  # 0% coupon rate
coupons_per_year_B3 = 1

# Calculate Macaulay Duration for each bond
duration_B1 = rk.macaulay_duration(rk.bond_cash_flows(maturity_B1, face_value, coupon_rate_B1, coupons_per_year_B1), yield_curve)
duration_B2 = rk.macaulay_duration(rk.bond_cash_flows(maturity_B2, face_value, coupon_rate_B2, coupons_per_year_B2), yield_curve)
duration_B3 = rk.macaulay_duration(rk.bond_cash_flows(maturity_B3, face_value, coupon_rate_B3, coupons_per_year_B3), yield_curve)

# Liabilities durations (from Question 7)
liabilities_durations = [3, 5, 10]

# Check combinations of pairs
pairs = [(duration_B1, duration_B2), (duration_B1, duration_B3), (duration_B2, duration_B3)]

for pair in pairs:
    duration_sum = pair[0] + pair[1]
    matched = any(abs(duration_sum - l) < 1e-5 for l in liabilities_durations)
    if not matched:
        print(f"The pair with durations {pair} cannot be used to build a duration matched portfolio.")


The pair with durations (18.74518415380995, 16.32082218369695) cannot be used to build a duration matched portfolio.
The pair with durations (18.74518415380995, 10.0) cannot be used to build a duration matched portfolio.
The pair with durations (16.32082218369695, 10.0) cannot be used to build a duration matched portfolio.


In [59]:
import numpy as np

# Define the bonds
bond_b2 = {'face_value': 1000, 'years_to_maturity': 5, 'coupon_rate': 0.06, 'coupons_per_year': 4}
bond_b3 = {'face_value': 1000, 'years_to_maturity': 10, 'coupon_rate': 0.0, 'coupons_per_year': 1}

# Define the liabilities
liabilities = np.array([100000, 200000, 300000])
years_to_maturity = np.array([3, 5, 10])

# Calculate the cash flows of the bonds
def calculate_cash_flows(bond):
    face_value = bond['face_value']
    years_to_maturity = bond['years_to_maturity']
    coupon_rate = bond['coupon_rate']
    coupons_per_year = bond['coupons_per_year']
    cash_flows = np.zeros(int(years_to_maturity * coupons_per_year))
    for i in range(int(years_to_maturity * coupons_per_year)):
        if i % (coupons_per_year) == 0 and i!= 0:
            cash_flows[i] = face_value * coupon_rate / coupons_per_year
        if i == int(years_to_maturity * coupons_per_year) - 1:
            cash_flows[i] += face_value
    return cash_flows

cash_flows_b2 = calculate_cash_flows(bond_b2)
cash_flows_b3 = calculate_cash_flows(bond_b3)

# Calculate the prices of the bonds
def calculate_price(bond, cash_flows, yield_to_maturity):
    return np.sum(cash_flows / (1 + yield_to_maturity / bond['coupons_per_year']) ** np.arange(1, len(cash_flows) + 1))

yield_to_maturity = 0.05
price_b2 = calculate_price(bond_b2, cash_flows_b2, yield_to_maturity)
price_b3 = calculate_price(bond_b3, cash_flows_b3, yield_to_maturity)

# Calculate the durations of the bonds
def calculate_duration(bond, cash_flows, price, yield_to_maturity):
    duration = np.sum((np.arange(1, len(cash_flows) + 1) * cash_flows / (1 + yield_to_maturity / bond['coupons_per_year']) ** np.arange(1, len(cash_flows) + 1)) / price)
    return duration / bond['coupons_per_year']

duration_b2 = calculate_duration(bond_b2, cash_flows_b2, price_b2, yield_to_maturity)
duration_b3 = calculate_duration(bond_b3, cash_flows_b3, price_b3, yield_to_maturity)

# Calculate the duration of the liabilities
duration_liabilities = np.sum(liabilities * years_to_maturity) / np.sum(liabilities)

# Calculate the weights of the bonds in the portfolio
weight_b2 = (duration_b3 - duration_liabilities) / (duration_b3 - duration_b2)
weight_b3 = 1 - weight_b2

print("The weight of B2 in the portfolio is:", weight_b2)

The weight of B2 in the portfolio is: 0.5506333430613988
